# Implementación diversos algortimos  Clasificación Multiclase

In [1]:
# Librerias y dependencias
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial import distance
from sklearn.metrics import f1_score, accuracy_score
import pycaret
from pycaret.classification import setup
from pycaret.classification import compare_models
import sys
import math


In [2]:

GC1 = pd.read_excel("../Archivos Generados/PipelineResults/GruposCalidad-Fase2/Grupo1.xlsx",index_col=0)
GC2 = pd.read_excel("../Archivos Generados/PipelineResults/GruposCalidad-Fase2/Grupo2.xlsx",index_col=0)
GC3 = pd.read_excel("../Archivos Generados/PipelineResults/GruposCalidad-Fase2/Grupo3.xlsx",index_col=0)

In [3]:
print(f"Grupo de Calidad 1 {len(GC1)}, Grupo de Calidad 2: {len(GC2)}, Grupo de Calidad 3: {len(GC3)}")

Grupo de Calidad 1 398, Grupo de Calidad 2: 84, Grupo de Calidad 3: 317


In [4]:
# Se define la etiqueta de c/ grupo
GC1["Grupo"] = 1
GC2["Grupo"] = 2
GC3["Grupo"] = 3


In [6]:
# Se Unen los diferentes Grupos en un unico dataset
df = pd.concat([GC1, GC2,GC3], ignore_index=True)

In [7]:
df.shape

(799, 177)

In [8]:
# Se eliminan algunas variables no relevantes
df = df.drop(["ID_LOTE", "RDT_AJUSTADO"], axis=1)

In [9]:
df.head(5)

,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,ContMalMec_Flor_Cose,...,MOTEADOS_RASTA,MOTEADOS_MAS70cm._RASTA,OBSERVA_EROSION_RASTA,OBSERVA_MOHO_RASTA,OBSERVA_RAICES_VIVAS_RASTA,OBSERVA_HOJARASCA_MO_RASTA,SUELO_NEGRO_BLANDO_RASTA,CUCHILLO_PRIMER_HTE_RASTA,CERCA_RIOS_QUEBRADAS_RASTA,Grupo
0,6,45,77,54000,11,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
1,5,54,80,60000,10,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
2,4,48,82,55000,12,1,0,0,0,0,...,1,0,0,0,1,0,0,1,0,1
3,5,47,73,70000,72,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
4,5,47,81,67000,14,1,0,0,0,0,...,0,0,0,0,1,1,0,1,0,1


In [10]:
df.tail(5)

,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,ContMalMec_Flor_Cose,...,MOTEADOS_RASTA,MOTEADOS_MAS70cm._RASTA,OBSERVA_EROSION_RASTA,OBSERVA_MOHO_RASTA,OBSERVA_RAICES_VIVAS_RASTA,OBSERVA_HOJARASCA_MO_RASTA,SUELO_NEGRO_BLANDO_RASTA,CUCHILLO_PRIMER_HTE_RASTA,CERCA_RIOS_QUEBRADAS_RASTA,Grupo
794,6,50,75,60000,17,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,3
795,5,48,88,60000,17,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,3
796,6,49,87,60000,17,0,0,0,0,0,...,0,0,0,0,1,1,0,1,0,3
797,7,47,84,70000,18,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,3
798,6,49,79,61000,17,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,3


## Pycaret
-  Ref: https://machinelearningmastery.com/pycaret-for-machine-learning/

In [11]:
print('PyCaret: %s' % pycaret.__version__)

PyCaret: 3.1.0


In [12]:
df

,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,ContMalMec_Flor_Cose,...,MOTEADOS_RASTA,MOTEADOS_MAS70cm._RASTA,OBSERVA_EROSION_RASTA,OBSERVA_MOHO_RASTA,OBSERVA_RAICES_VIVAS_RASTA,OBSERVA_HOJARASCA_MO_RASTA,SUELO_NEGRO_BLANDO_RASTA,CUCHILLO_PRIMER_HTE_RASTA,CERCA_RIOS_QUEBRADAS_RASTA,Grupo
0,6,45,77,54000,11,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
1,5,54,80,60000,10,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
2,4,48,82,55000,12,1,0,0,0,0,...,1,0,0,0,1,0,0,1,0,1
3,5,47,73,70000,72,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
4,5,47,81,67000,14,1,0,0,0,0,...,0,0,0,0,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,6,50,75,60000,17,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,3
795,5,48,88,60000,17,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,3
796,6,49,87,60000,17,0,0,0,0,0,...,0,0,0,0,1,1,0,1,0,3
797,7,47,84,70000,18,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,3


In [21]:
# setup the dataset
grid = setup(data=df, target=df.columns[-1])

,Description,Value
0,Session id,3266
1,Target,Grupo
2,Target type,Multiclass
3,Target mapping,"1: 0, 2: 1, 3: 2"
4,Original data shape,"(799, 175)"
5,Transformed data shape,"(799, 175)"
6,Transformed train set shape,"(559, 175)"
7,Transformed test set shape,"(240, 175)"
8,Numeric features,174
9,Preprocess,True


In [22]:
# evaluate models and compare models
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.5419,0.5925,0.4213,0.5007,0.4348,0.1664,0.1733,0.1340
rf,Random Forest Classifier,0.5241,0.5731,0.3862,0.4242,0.3908,0.1262,0.1322,0.1370
catboost,CatBoost Classifier,0.5150,0.5682,0.3769,0.4355,0.3933,0.1140,0.1190,11.6490
gbc,Gradient Boosting Classifier,0.5044,0.5784,0.3774,0.4017,0.3843,0.1031,0.1068,0.7250
dummy,Dummy Classifier,0.4973,0.5000,0.1000,0.0500,0.0667,0.0000,0.0000,0.0230
lr,Logistic Regression,0.4867,0.5218,0.3030,0.3565,0.3221,0.0386,0.0421,1.4410
lightgbm,Light Gradient Boosting Machine,0.4864,0.5628,0.3841,0.4001,0.3808,0.0689,0.0706,0.3430
knn,K Neighbors Classifier,0.4526,0.4959,0.3364,0.3770,0.3435,0.0025,0.0019,0.0520
ridge,Ridge Classifier,0.4474,0.0000,0.3223,0.3401,0.3277,-0.0048,-0.0041,0.0240
ada,Ada Boost Classifier,0.4438,0.5299,0.3476,0.3657,0.3500,0.0073,0.0082,0.1010


In [23]:
# Reporte del mejor modelo
print(best)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     n_estimators=100, n_jobs=-1, oob_score=False,
                     random_state=3266, verbose=0, warm_start=False)


**Nota** Teniendo en cuenta el dataset suministrado la mejor presición que se obtiene es con el algoritmo Extra Tree Classificaer

In [ ]:
# Ajuste de hiperparametros para el mejor modelo
